# Analiza: Wykrywanie języka na podstawie częstotliwości słów w tekscie

Celem tej pracy jest ocena, czy na podstawie danych dotyczących częstotliwości występowania słów w arytkule na wybranym wiki (u nas bulbapedia) da się rozpoznać język tekstu. Eksperyment polega na porównaniu listy najczęsciej występujących słów na wiki z listą najczęściej występujących słów w jednym z trzech języków. Sprawdzimy dodatkowo jak na poprawność metody wpływa długość arytkułu.

In [4]:
# Początkowe importy najważniejszych modułów i bibliotek
import json
import math
import matplotlib.pyplot as plt
from wordfreq import top_n_list, zipf_frequency
from pathlib import Path
from scraper_logic import Scraper

Wykorzystamy listy najczęsciej występujących słów z trzech języków: polskiego, angielskiego (język naszej wiki), oraz niemieckiego. Użyjemy do tego biblioteki wordfreq.

In [5]:
def get_language_words(lang, n=1000):
    words = top_n_list(lang, n)
    return {
        w : zipf_frequency(w, lang)
        for w in words 
    }
    
en_dict = get_language_words('en')
pl_dict = get_language_words('pl')
de_dict = get_language_words('de')

Do pobrania słów z arytkułów z wiki użyjemy pliku tworzonego wywołaniem funkcji scrapera do_count_words(), która aktualizuje plik o słownik [słowa : liczba wystąpień] dla danej frazy. 

In [6]:
def read_word_file(): 
    with open('./word-counts.json', 'r', encoding='utf-8') as f:
        return json.load(f)
    
def get_words(phrase, base_url='https://bulbapedia.bulbagarden.net/wiki/'):
    # Tworzymy instancje scrapera dla danej frazy.
    scraper = Scraper(phrase, base_url)

    # Usuwamy zawartość pliku word-count.json
    try:
        Path('./word-counts.json').unlink()
    except FileNotFoundError:
        pass
    
    # Liczymy słowa dla danego artykułu
    scraper.do_count_words()

    # Zwracamy otrzymany słownik
    return read_word_file()

Przygotujmy i zapiszmy do zmiennej dane z długiego artykułu. Tworzymy dedykowany plik word-analysis.json, używamy scrapera do zapełnienia go słowami z danego artykułu i zapisujemy z użyciem powyższej funkcji. Jako długi arytykuł użyjemy opisu regionu Paldea z bulbapedii [https://bulbapedia.bulbagarden.net/wiki/Paldea].

In [10]:
long_article = get_words('Paldea')

Zrobimy to samo dla krótkiego arytkuły ze strony [https://bulbapedia.bulbagarden.net/wiki/Bulbakaki], który pełen jest nazw własnych.

In [9]:
short_article = get_words('Bulbakaki')

Musimy zrobić to samo dla artykułów w każdym z wybranych języków do czego również użyjemy scrapera ale z podmienionym base_url, skorzystamy ze strony wikipedii     

In [ ]:
    pl_article = get_words('Informatyka', 'https://pl.wikipedia.org/wiki/')
print(pl)

ArticleNotFoundError: Could not find article content area.